# FM 24 Moneyball

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import re

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show all columns in pandas
pd.set_option('display.max_columns', 500) 

%matplotlib inline

## Ingest and clean data

In [2]:
### Read raw data

# Select file path for raw data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM 24 exports\Winter 2023\Center Backs - SL.html"

# Ingest in a dataframe
df_players_raw = pd.read_html(path, encoding='utf-8')[0]

# Format column names 
df_players_raw.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players_raw.columns]

# ---------------------------------------------------------------------------------------------------------------------

### Read processed Data lab data

# Select file path for processed data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM Data Lab outputs\Winter 2023\Center Backs - SL.csv"

# Ingest in a dataframe 
df_players = pd.read_csv(path)   

# Format column names 
df_players.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players.columns]

# Bring ID from raw data
df_players = df_players.merge(df_players_raw[['name', 'age', 'height', 'weight', 'uid']], 
                 left_on=['name', 'age', 'height', 'weight'], 
                 right_on=['name', 'age', 'height', 'weight']).reset_index().rename(columns={'level_0': 'id'})

df_players['id'] = df_players['uid']

df_players.drop(columns='uid', inplace=True)

## Data cleaning

In [3]:
# Get transfer valuation estimation
def parse_transfer_value(val):
    if pd.isnull(val):
        return None

    # Remove dollar signs and spaces
    val = val.replace('$', '').replace(' ', '')

    # If it's a range like "850K-8.4M"
    if '-' in val:
        low_str, high_str = val.split('-')
        low = parse_single_value(low_str)
        high = parse_single_value(high_str)
        if low is not None and high is not None:
            return (low + high) * (6/10)
    else:
        return parse_single_value(val)

    return None  # fallback

def parse_single_value(s):
    try:
        if s.endswith('M'):
            return float(s[:-1]) * 1_000_000
        elif s.endswith('K'):
            return float(s[:-1]) * 1_000
        else:
            return float(s)  # just in case it's a plain number
    except:
        return None

# Apply to your DataFrame
df_players['price_estimate_M'] = df_players['transfer_value'].apply(parse_transfer_value) / 1000000

df_players.head()

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,pressures_attempted,pressures_attempted/90,pressures_completed,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M
0,2000226646,Stijn Bultman,D (RC),18,187 cm,80 kg,Sct,Heracles Almelo,Eredivisie,NED,-,-,Scouting Required,"$1,100 p/w",$275K - $10M,-,Right Only,15,1321,6.94,1,82.56,0,84.21,78.55,93,7.15,8.50,0.14,0.16,0.07,0.08,46,3.13,19,1.29,41.30,9.13,6.20,2.93,47.26,1.16,0.89,76,0.27,0.00,1.13,1.02,0.61,0.55,5.86,4.22,72.09,1.64,27.91,0.82,5.86,72.82,0.00,1.02,1.57,0.25,0.14,15.19,0.48,20,100,0,47,53,13,13,20,7,13,0,13,13,0,0,6.165
1,2000178532,Tobias Slotsager,D (RC),17,190 cm,82 kg,Wnt,OB,3F Superliga,DEN,-,-,Scouting Required,"$3,800 p/w",$250K - $11.5M,-,Right,15,1344,6.95,1,84.00,1,80.22,75.00,93,5.02,6.26,0.27,0.33,0.27,0.33,47,3.15,16,1.07,34.04,13.06,5.56,7.50,134.89,0.87,0.60,69,0.27,0.33,3.25,1.67,0.74,0.60,7.43,6.29,84.68,1.14,15.32,1.07,7.43,83.06,0.07,0.27,0.94,0.40,0.33,21.39,0.67,40,100,20,27,40,53,60,47,47,60,40,60,67,53,13,7.050
2,2000163579,Aleksander Hammer Kjelsen,D (C),17,185 cm,78 kg,Wnt,Vålerenga,Eliteserien,NOR,-,Fairly Determined,Level-headed,$900 p/w,$210K - $2.1M,-,Right,30,2691,6.95,0,89.70,2,53.75,47.89,89,4.25,7.90,0.33,0.62,0.33,0.62,96,3.21,38,1.27,39.58,12.88,4.88,8.00,163.93,1.40,1.10,79,0.30,0.30,2.80,1.64,0.27,0.27,8.66,6.69,77.22,1.97,22.78,0.90,8.66,77.41,0.10,0.50,1.47,0.25,0.37,20.58,1.10,47,100,13,13,47,7,0,0,13,7,13,0,80,40,47,1.386
3,2000182819,Leny Yoro,D (C),18,191 cm,86 kg,-,Lille,Ligue 1 Uber Eats,FRA,-,Resolute,Level-headed,"$23,500 p/w",$91M - $115M,-,Right,10,1124,6.83,12,51.09,3,81.83,77.35,95,4.88,5.97,0.08,0.10,0.08,0.10,43,3.44,15,1.20,34.88,12.17,4.80,7.37,153.54,1.44,1.04,72,0.40,0.48,2.25,1.52,0.64,0.64,7.53,5.69,75.53,1.84,24.47,0.40,7.53,75.00,0.08,0.64,0.80,0.53,0.16,17.60,0.80,33,100,40,40,20,33,53,60,33,27,33,40,20,7,13,123.600
4,27161052,Alexander Busch,D (C),20,190 cm,75 kg,-,Silkeborg IF,3F Superliga,DEN,-,Balanced,Media-friendly,"$5,000 p/w",$5.6M - $8.2M,-,Right,21,1819,6.80,0,86.62,4,53.73,48.34,90,4.21,7.83,0.35,0.64,0.35,0.64,74,3.66,21,1.04,28.38,12.77,5.89,6.88,116.81,1.53,1.24,81,0.30,0.49,2.38,2.13,0.49,0.35,8.31,6.28,75.60,2.03,24.40,0.89,8.31,76.38,0.05,0.64,1.73,0.57,0.45,22.87,1.53,60,100,73,53,60,60,33,40,53,47,60,80,60,73,80,8.280


## Plot function

In [4]:
def scatter_plot(df, x_metric, y_metric, z_metric, 
                 title, show_color_bar=False, show_name_label=True): 
    # Set text to 'name' if show_name_label is True, else empty string
    text_col = 'name' if show_name_label else None

    # Create plotly figure
    fig = px.scatter(
        df,
        x=x_metric,
        y=y_metric,
        color=z_metric,
        text=text_col,
        color_continuous_scale='RdYlGn',
        labels={z_metric: f'Color Scale ({z_metric})'},
        title=title,
        width=1200, 
        height=700    
    )

    # Format traces
    fig.update_traces(
        marker=dict(size=10), 
        textposition='top center',
        textfont=dict(size=10),
        customdata=df[['name', 'club']],  # Always pass 'name' for hovertemplate
        hovertemplate=(
            "<b>%{customdata[0]}</b><br>" +
            "<b>%{customdata[1]}</b><br><br>" +
            f"{x_metric}: " + "%{x}<br>" +
            f"{y_metric}: " + "%{y}<br>" +
            f"{z_metric}: " + "%{marker.color}<extra></extra>"
        )
    )

    fig.update_layout(
        title={
            'text': f'{title} ({len(df): ,.0f})',
            'x': 0.5,
            'xanchor': 'center'
        },
        coloraxis_colorbar=dict(title=z_metric),
        coloraxis_showscale=show_color_bar,  # Show or hide the color bar
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False),
        yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False)
    )

    fig.show()


## Passing analysis

### Full universe

In [37]:
# Initialize list of targets
targets = df_players['id']

scatter_plot(df_players,
             x_metric='passes_attempted/90', 
             y_metric='progressive_pass_rate',
             z_metric='pass_completion_%',
             title='Passing quality', 
             show_name_label=True
)

In [38]:
# Players to drop
drop_list = [
    2000182819, 
    16147700
]

df_players = df_players[~df_players['id'].isin(drop_list)]

df_players[df_players['name'].str.contains('Yoro')]

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,pressures_attempted,pressures_attempted/90,pressures_completed,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M


## Possession Analysis

In [39]:
# Full universe 
scatter_plot(df_players,
             x_metric='possession_lost/90', 
             y_metric='possession_won/90',
             z_metric='poss+_/90',
             title='Possesion quality', 
             show_name_label=True
)

In [40]:
# Players to drop
drop_list = [
    2000182819, # Wober - Not progressive enough / Age
    16147700, # Yoro - Not progressive enough / High price
    49048468, # Matsima - Propensity to lose the ball
]

df_players = df_players[~df_players['id'].isin(drop_list)]

df_players[df_players['name'].str.contains('Mats')]

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,pressures_attempted,pressures_attempted/90,pressures_completed,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M


## Aereal/Ground Dominance 

In [41]:
# Full universe 
scatter_plot(df_players,
             x_metric='tackle_completion_%', 
             y_metric='headers_won_%',
             z_metric='duels_win_%',
             title='Dominance', 
             show_name_label=True
)

In [42]:
# Players to drop
drop_list = [
    2000182819, # Wober - Not progressive enough / Age
    16147700, # Yoro - Not progressive enough / High price
    49048468, # Matsima - Propensity to lose the ball
    83320135, # Silva - Lack of aereal dominance
    16130398, # Lienhart - Age / Tackle dominance / Price
    12094339, # Sylla - Price
]

df_players = df_players[~df_players['id'].isin(drop_list)]

df_players[df_players['name'].str.contains('Lie')]

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,pressures_attempted,pressures_attempted/90,pressures_completed,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M


## Pressure quality

In [43]:
# Full universe 
scatter_plot(df_players,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

In [44]:
# Full universe 
scatter_plot(df_players,
             x_metric='interceptions/90', 
             y_metric='blocks/90',
             z_metric='clearances/90',
             title='Presence', 
             show_name_label=True
)

In [45]:
df_players.head(10)

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,pressures_attempted,pressures_attempted/90,pressures_completed,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M
0,2000226646,Stijn Bultman,D (RC),18,187 cm,80 kg,Sct,Heracles Almelo,Eredivisie,NED,-,-,Scouting Required,"$1,100 p/w",$275K - $10M,-,Right Only,15,1321,6.94,1,82.56,0,84.21,78.55,93,7.15,8.50,0.14,0.16,0.07,0.08,46,3.13,19,1.29,41.30,9.13,6.20,2.93,47.26,1.16,0.89,76,0.27,0.00,1.13,1.02,0.61,0.55,5.86,4.22,72.09,1.64,27.91,0.82,5.86,72.82,0.00,1.02,1.57,0.25,0.14,15.19,0.48,20,100,0,47,53,13,13,20,7,13,0,13,13,0,0,6.165
1,2000178532,Tobias Slotsager,D (RC),17,190 cm,82 kg,Wnt,OB,3F Superliga,DEN,-,-,Scouting Required,"$3,800 p/w",$250K - $11.5M,-,Right,15,1344,6.95,1,84.00,1,80.22,75.00,93,5.02,6.26,0.27,0.33,0.27,0.33,47,3.15,16,1.07,34.04,13.06,5.56,7.50,134.89,0.87,0.60,69,0.27,0.33,3.25,1.67,0.74,0.60,7.43,6.29,84.68,1.14,15.32,1.07,7.43,83.06,0.07,0.27,0.94,0.40,0.33,21.39,0.67,40,100,20,27,40,53,60,47,47,60,40,60,67,53,13,7.050
2,2000163579,Aleksander Hammer Kjelsen,D (C),17,185 cm,78 kg,Wnt,Vålerenga,Eliteserien,NOR,-,Fairly Determined,Level-headed,$900 p/w,$210K - $2.1M,-,Right,30,2691,6.95,0,89.70,2,53.75,47.89,89,4.25,7.90,0.33,0.62,0.33,0.62,96,3.21,38,1.27,39.58,12.88,4.88,8.00,163.93,1.40,1.10,79,0.30,0.30,2.80,1.64,0.27,0.27,8.66,6.69,77.22,1.97,22.78,0.90,8.66,77.41,0.10,0.50,1.47,0.25,0.37,20.58,1.10,47,100,13,13,47,7,0,0,13,7,13,0,80,40,47,1.386
4,27161052,Alexander Busch,D (C),20,190 cm,75 kg,-,Silkeborg IF,3F Superliga,DEN,-,Balanced,Media-friendly,"$5,000 p/w",$5.6M - $8.2M,-,Right,21,1819,6.80,0,86.62,4,53.73,48.34,90,4.21,7.83,0.35,0.64,0.35,0.64,74,3.66,21,1.04,28.38,12.77,5.89,6.88,116.81,1.53,1.24,81,0.30,0.49,2.38,2.13,0.49,0.35,8.31,6.28,75.60,2.03,24.40,0.89,8.31,76.38,0.05,0.64,1.73,0.57,0.45,22.87,1.53,60,100,73,53,60,60,33,40,53,47,60,80,60,73,80,8.280
6,19232517,Pedro Henrique,D (C),28,188 cm,82 kg,Wnt,Athlético Paranaense,Campeonato Brasileiro Série A Assaí,BRA,-,-,Scouting Required,"$9,000 p/w",$9.2M - $11M,-,Right,44,3775,7.03,2,82.07,6,57.22,51.52,90,3.58,6.25,0.21,0.38,0.21,0.38,175,4.17,70,1.67,40.00,14.73,4.22,10.51,249.05,1.72,1.41,82,0.31,0.36,4.50,1.79,0.48,0.31,7.77,6.10,78.53,1.67,21.47,1.26,7.77,79.15,0.14,0.38,1.05,0.81,0.36,23.90,1.79,53,100,53,27,33,67,73,67,67,67,73,87,73,80,67,12.120
8,91144914,Kevin Danso,D (C),25,190 cm,85 kg,Wnt,Lens,Ligue 1 Uber Eats,AUT (GHA),Trained in nation (0-21),-,Scouting Required,"$21,500 p/w",$29M - $45M,-,Right,20,1622,7.08,0,81.10,8,59.98,54.16,90,4.27,7.12,0.11,0.19,0.11,0.19,69,3.83,23,1.28,33.33,12.10,7.10,5.00,70.42,1.83,1.22,67,0.61,0.22,2.20,2.05,0.94,0.78,7.82,6.33,80.85,1.50,19.15,1.50,7.82,78.16,0.17,0.83,2.50,0.38,0.11,26.61,0.89,7,100,60,20,0,20,20,13,20,20,27,67,100,87,13,44.400
12,49038967,Bafodé Diakité,D (RC),22,185 cm,74 kg,Sct,Lille,Ligue 1 Uber Eats,FRA (GUI),-,-,Scouting Required,"$27,000 p/w",$14.5M - $18.5M,-,Right,

In [46]:
df_players.groupby('division')['id'].count()

division
3F Superliga                           2
Campeonato Brasileiro Série A Assaí    1
Eliteserien                            1
Eredivisie                             1
Ligue 1 Uber Eats                      3
Serie A TIM                            1
Name: id, dtype: int64

In [47]:
df_players.groupby('division').get_group('Campeonato Brasileiro Série A Assaí').sort_values('age')

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,pressures_attempted,pressures_attempted/90,pressures_completed,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,aerial_challenges_attempted/90,duels_win_%,fouls_won/90,fouls_committed/90,clearances/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,goalkeeping,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M
6,19232517,Pedro Henrique,D (C),28,188 cm,82 kg,Wnt,Athlético Paranaense,Campeonato Brasileiro Série A Assaí,BRA,-,-,Scouting Required,"$9,000 p/w",$9.2M - $11M,-,Right,44,3775,7.03,2,82.07,6,57.22,51.52,90,3.58,6.25,0.21,0.38,0.21,0.38,175,4.17,70,1.67,40.0,14.73,4.22,10.51,249.05,1.72,1.41,82,0.31,0.36,4.5,1.79,0.48,0.31,7.77,6.1,78.53,1.67,21.47,1.26,7.77,79.15,0.14,0.38,1.05,0.81,0.36,23.9,1.79,53,100,53,27,33,67,73,67,67,67,73,87,73,80,67,12.12
